In [1]:
import re
import nltk
import string
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gensim

pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [2]:
df = pd.read_csv("./dataforw2v.csv")

In [3]:
df.head()

,Unnamed: 0,text,cleaned_text
0,0,Y bitcoin,y bitcoin
1,1,Prices update in $USD (1 hour): $BTC - 9377.1 $(+0.14 %) $ETH - 183.41 $(+0.29 %) $XRP - 0.24 $(-0.07 %) $BCH - 379.5 $(+0.42 %) $LTC - 70.9 $(+0.4 %) #BTC #ETH #XRP #BCH #LTC #Trading #Crypto #B...,prices update in usd hour btc eth xrp bch ltc
2,2,Would love 250/300k for one Bitcoin #2022 $BTC,would love k for one bitcoin btc
3,3,Once #Bitcoin hits the ATH 100% of days will have been profitable. https://www.lookintobitcoin.com/charts/bitcoin-profitable-days/,once hits the ath of days will have been profitable profitable days
4,4,Can we exchange #bitcoin for fiat?Or your services is for purchase of #bitcoin &amp; #cryptocurrency,can we exchange for fiat or your services is for purchase of amp


In [6]:
df['cleaned_text'] = df['cleaned_text'].astype(str)

In [7]:
df.cleaned_text = df.cleaned_text.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
df.head(10)

,Unnamed: 0,text,cleaned_text
0,0,Y bitcoin,bitcoin
1,1,Prices update in $USD (1 hour): $BTC - 9377.1 $(+0.14 %) $ETH - 183.41 $(+0.29 %) $XRP - 0.24 $(-0.07 %) $BCH - 379.5 $(+0.42 %) $LTC - 70.9 $(+0.4 %) #BTC #ETH #XRP #BCH #LTC #Trading #Crypto #B...,prices update hour
2,2,Would love 250/300k for one Bitcoin #2022 $BTC,would love bitcoin
3,3,Once #Bitcoin hits the ATH 100% of days will have been profitable. https://www.lookintobitcoin.com/charts/bitcoin-profitable-days/,once hits days will have been profitable profitable days
4,4,Can we exchange #bitcoin for fiat?Or your services is for purchase of #bitcoin &amp; #cryptocurrency,exchange fiat your services purchase
5,5,Bitcoin price index https://www.worldcoinindex.com/coin/bitcoin #USD #EUR #CNY #GBP #RUB,bitcoin price index
6,6,Ethereum co-founder Vitalik Buterin’s relation with Bitcoin Cash https://coinemama.com/ethereum-co-founder-vitalik-buterins-relation-with-bitcoin-cash/,ethereum founder vitalik buterin relation with bitcoin cash founder vitalik buterins relation with bitcoin cash
7,7,However bitcoin is ratpoison! These people have NFI!,however bitcoin ratpoison these people have
8,8,The amount of potential set ups is suspect.. suspicious. #Bitcoin #Crypto,amount potential suspect suspicious
9,9,And I personally thought it'd be good to bring it up along with bitcoin for Libertarians and people who only like gold. I did that a few times years ago but I've lost contact with those people.,personally thought good bring along with bitcoin libertarians people only like gold that times years lost contact with those people


In [8]:
tokenized_tweet = df.cleaned_text.apply(lambda x: x.split())
tokenized_tweet.head()

0                                                             [bitcoin]
1                                                [prices, update, hour]
2                                                [would, love, bitcoin]
3    [once, hits, days, will, have, been, profitable, profitable, days]
4                            [exchange, fiat, your, services, purchase]
Name: cleaned_text, dtype: object

In [9]:
from nltk.stem.porter import * 
stemmer = PorterStemmer() 
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])
tokenized_tweet.head()

0                                                 [bitcoin]
1                                      [price, updat, hour]
2                                    [would, love, bitcoin]
3    [onc, hit, day, will, have, been, profit, profit, day]
4                    [exchang, fiat, your, servic, purchas]
Name: cleaned_text, dtype: object

In [10]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])    
df['cleaned_text'] = tokenized_tweet
df.head(10)

,Unnamed: 0,text,cleaned_text
0,0,Y bitcoin,bitcoin
1,1,Prices update in $USD (1 hour): $BTC - 9377.1 $(+0.14 %) $ETH - 183.41 $(+0.29 %) $XRP - 0.24 $(-0.07 %) $BCH - 379.5 $(+0.42 %) $LTC - 70.9 $(+0.4 %) #BTC #ETH #XRP #BCH #LTC #Trading #Crypto #B...,price updat hour
2,2,Would love 250/300k for one Bitcoin #2022 $BTC,would love bitcoin
3,3,Once #Bitcoin hits the ATH 100% of days will have been profitable. https://www.lookintobitcoin.com/charts/bitcoin-profitable-days/,onc hit day will have been profit profit day
4,4,Can we exchange #bitcoin for fiat?Or your services is for purchase of #bitcoin &amp; #cryptocurrency,exchang fiat your servic purchas
5,5,Bitcoin price index https://www.worldcoinindex.com/coin/bitcoin #USD #EUR #CNY #GBP #RUB,bitcoin price index
6,6,Ethereum co-founder Vitalik Buterin’s relation with Bitcoin Cash https://coinemama.com/ethereum-co-founder-vitalik-buterins-relation-with-bitcoin-cash/,ethereum founder vitalik buterin relat with bitcoin cash founder vitalik buterin relat with bitcoin cash
7,7,However bitcoin is ratpoison! These people have NFI!,howev bitcoin ratpoison these peopl have
8,8,The amount of potential set ups is suspect.. suspicious. #Bitcoin #Crypto,amount potenti suspect suspici
9,9,And I personally thought it'd be good to bring it up along with bitcoin for Libertarians and people who only like gold. I did that a few times years ago but I've lost contact with those people.,person thought good bring along with bitcoin libertarian peopl onli like gold that time year lost contact with those peopl


In [34]:
%%time

tokenized_tweet = df['cleaned_text'].apply(lambda x: x.split()) # tokenizing 

model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            size=200, # output dimensions
            window=5, # context window size
            min_count=50, # ignore any words that appear less than 50 times.                                  
            sg = 1, # skip-gram model
            hs = 0,
            negative = 5, # negative sampling
            workers= 12, # number of cores that can be used
) 

model_w2v.train(tokenized_tweet, total_examples= len(df['cleaned_text']), epochs=15)

Wall time: 17min 27s


(704425350, 820215645)

In [41]:
model_w2v.wv.most_similar(positive="economi")

[('econom', 0.5518285036087036),
 ('societi', 0.5289602279663086),
 ('environ', 0.4842783212661743),
 ('monetari', 0.4699172377586365),
 ('hyperinfl', 0.4675235450267792),
 ('devalu', 0.4548271596431732),
 ('econ', 0.44990524649620056),
 ('govt', 0.4496572017669678),
 ('debt', 0.4484199285507202),
 ('gov', 0.4466349482536316)]

In [42]:
word_vectors = model_w2v.wv

In [43]:
word_vectors.save('./trainedwordvectors.kv')

In [44]:
model_w2v.save('./model_w2v.model')

In [45]:
print(model_w2v['china'])

[ 1.38593063e-01 -1.68801427e-01 -1.86792370e-02  7.44740441e-02
  1.49935678e-01 -1.50406331e-01  2.08930388e-01 -2.13830933e-01
 -5.18644333e-01  1.88223988e-01 -3.18639904e-01  2.87007689e-01
 -7.15638697e-02  1.01785241e-02  4.10099998e-02  4.83288437e-01
  1.45913959e-01  3.34703118e-01 -3.55140418e-01 -3.67752731e-01
 -2.73817599e-01  8.71907845e-02  6.21754527e-02  4.63143177e-03
 -9.43851024e-02  9.69165936e-02  6.31797016e-02  1.40076160e-01
 -5.40537775e-01  1.82207916e-02  3.98640692e-01 -1.17308728e-01
 -6.12851121e-02  1.69392169e-01 -4.02759761e-01 -3.09553593e-01
 -2.71309733e-01  1.00915901e-01  1.16313823e-01  1.52991697e-01
  4.23344910e-01  2.16856584e-01 -5.30343354e-01 -2.24792197e-01
  1.71728730e-01  3.50007534e-01 -4.20321018e-01  3.38185638e-01
  2.89284468e-01  1.53420910e-01 -5.13218865e-02  2.42017418e-01
  9.56411809e-02 -2.18375593e-01 -1.32159978e-01  2.70435899e-01
 -1.75274145e-02  6.37846068e-03  8.45731273e-02  4.49221916e-02
 -2.30792612e-01  1.49083